In [1]:
from data_load import mixture
import tensorflow as tf
import numpy as np
path = '/Users/smaity/projects/mnist-label-shift/MNIST'

In [2]:
x_source, y_source = mixture(12000, 0.9, flatten=False)
x_target, y_target = mixture(1000, 0.2, flatten=False)
x_source.shape, y_source.shape

((12000, 28, 28), (12000,))

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(3, 2, input_shape=(28, 28)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(20),
  tf.keras.layers.Dense(2)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.fit(x_source, y_source, epochs=20)

Epoch 1/20
375/375 [==============================] - 1s 3ms/step - loss: 4.0238 - accuracy: 0.8888
Epoch 2/20
375/375 [==============================] - 1s 3ms/step - loss: 0.7613 - accuracy: 0.9281
Epoch 3/20
375/375 [==============================] - 1s 3ms/step - loss: 0.3656 - accuracy: 0.9455
Epoch 4/20
375/375 [==============================] - 1s 3ms/step - loss: 0.2103 - accuracy: 0.9550
Epoch 5/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1665 - accuracy: 0.9606
Epoch 6/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1236 - accuracy: 0.9656
Epoch 7/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1211 - accuracy: 0.9659
Epoch 8/20
375/375 [==============================] - 1s 3ms/step - loss: 0.0849 - accuracy: 0.9747
Epoch 9/20
375/375 [==============================] - 1s 3ms/step - loss: 0.0729 - accuracy: 0.9783
Epoch 10/20
375/375 [==============================] - 1s 3ms/step - loss: 0.0702 - accuracy: 0.9775

In [4]:
model.evaluate(x_target, y_target, verbose=2)

32/32 - 0s - loss: 0.9543 - accuracy: 0.8460


[0.9542704820632935, 0.8460000157356262]

In [5]:
y_predict = model.predict_classes(x_target)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [6]:
from sklearn import metrics
m = 12000
reg = 0
prop_source = 0.9
confusion_matrix = metrics.confusion_matrix(model.predict_classes(x_source),y_source, labels=[0,1])/m
prop_target = np.mean(model.predict_classes(x_target))
    
xi = np.array([1-prop_target,prop_target])
w = np.matmul(np.linalg.inv(confusion_matrix + reg*np.diag([1, 1])),xi)
prop_targets = w*np.array([1-prop_source, prop_source])
if prop_targets[0] <0:
    prop_targets[0] = 0
if prop_targets[1] <0:
    prop_targets[1] = 0
prop_targets = prop_targets/np.sum(prop_targets)

In [7]:
prop_targets

array([0.67731092, 0.32268908])

In [8]:
sample_weight_0 = prop_targets[0]/0.1
sample_weight_1 = prop_targets[1]/0.9
sample_weight = sample_weight_0 * (1-y_source) + sample_weight_1 * y_source


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(3, 2, input_shape=(28, 28)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(20),
  tf.keras.layers.Dense(2)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.fit(x_source, y_source, epochs=20, sample_weight = sample_weight)


Epoch 1/20
375/375 [==============================] - 0s 1ms/step - loss: 9.4257 - accuracy: 0.6656
Epoch 2/20
375/375 [==============================] - 0s 1ms/step - loss: 2.2766 - accuracy: 0.7797
Epoch 3/20
375/375 [==============================] - 0s 1ms/step - loss: 1.0326 - accuracy: 0.8269
Epoch 4/20
375/375 [==============================] - 0s 1ms/step - loss: 0.6629 - accuracy: 0.8606
Epoch 5/20
375/375 [==============================] - 0s 1ms/step - loss: 0.5728 - accuracy: 0.8674
Epoch 6/20
375/375 [==============================] - 0s 1ms/step - loss: 0.4340 - accuracy: 0.8827
Epoch 7/20
375/375 [==============================] - 0s 1ms/step - loss: 0.3701 - accuracy: 0.8884
Epoch 8/20
375/375 [==============================] - 0s 1ms/step - loss: 0.2384 - accuracy: 0.9116
Epoch 9/20
375/375 [==============================] - 0s 1ms/step - loss: 0.2152 - accuracy: 0.9204
Epoch 10/20
375/375 [==============================] - 0s 1ms/step - loss: 0.1732 - accuracy: 0.9281

In [9]:
model.evaluate(x_target, y_target, verbose=2)

32/32 - 0s - loss: 0.2875 - accuracy: 0.9330


[0.2875029742717743, 0.9330000281333923]

In [10]:
sample_weight_0 = 1/0.1
sample_weight_1 = 1/0.9
sample_weight = sample_weight_0 * (1-y_source) + sample_weight_1 * y_source


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(3, 2, input_shape=(28, 28)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(20),
  tf.keras.layers.Dense(2)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.fit(x_source, y_source, epochs=20, sample_weight = sample_weight)

Epoch 1/20
375/375 [==============================] - 1s 2ms/step - loss: 14.0383 - accuracy: 0.7500
Epoch 2/20
375/375 [==============================] - 1s 1ms/step - loss: 3.6988 - accuracy: 0.8206
Epoch 3/20
375/375 [==============================] - 1s 1ms/step - loss: 1.6394 - accuracy: 0.8620
Epoch 4/20
375/375 [==============================] - 0s 1ms/step - loss: 1.1288 - accuracy: 0.8804
Epoch 5/20
375/375 [==============================] - 0s 1ms/step - loss: 0.6927 - accuracy: 0.9076
Epoch 6/20
375/375 [==============================] - 0s 1ms/step - loss: 0.5505 - accuracy: 0.9187
Epoch 7/20
375/375 [==============================] - 0s 1ms/step - loss: 0.4572 - accuracy: 0.9246
Epoch 8/20
375/375 [==============================] - 0s 1ms/step - loss: 0.4456 - accuracy: 0.9280
Epoch 9/20
375/375 [==============================] - 0s 1ms/step - loss: 0.3594 - accuracy: 0.9389
Epoch 10/20
375/375 [==============================] - 0s 1ms/step - loss: 0.2835 - accuracy: 0.953

In [12]:
m = model.evaluate(x_target, y_target, verbose=2)

32/32 - 0s - loss: 0.5239 - accuracy: 0.8960


In [14]:
m[1]

0.8960000276565552

In [1]:
from mnist_label_shift import label_shift
label_shift()

Epoch 1/5
375/375 [==============================] - 1s 3ms/step - loss: 3.0128 - accuracy: 0.8976
Epoch 2/5
375/375 [==============================] - 1s 3ms/step - loss: 0.5870 - accuracy: 0.9340
Epoch 3/5
375/375 [==============================] - 1s 3ms/step - loss: 0.2595 - accuracy: 0.9488
Epoch 4/5
375/375 [==============================] - 1s 3ms/step - loss: 0.1772 - accuracy: 0.9557
Epoch 5/5
375/375 [==============================] - 1s 3ms/step - loss: 0.1444 - accuracy: 0.9624
375/375 - 0s - loss: 0.1071 - accuracy: 0.9693
32/32 - 0s - loss: 1.8304 - accuracy: 0.6740
Epoch 1/5
375/375 [==============================] - 0s 1ms/step - loss: 20.4946 - accuracy: 0.7370
Epoch 2/5
375/375 [==============================] - 0s 1ms/step - loss: 5.3985 - accuracy: 0.8228
Epoch 3/5
375/375 [==============================] - 0s 1ms/step - loss: 2.5510 - accuracy: 0.8577
Epoch 4/5
375/375 [==============================] - 0s 1ms/step - loss: 1.6725 - accuracy: 0.8779
Epoch 5/5
375/37

(0.6740000247955322,
 0.8130000233650208,
 0.8410000205039978,
 0.9692500233650208,
 0.9023333191871643,
 0.8863333463668823,
 0.9620000123977661)

In [3]:
import itertools
n_targets = [1000, 5000, 10000]
pi_targets = [0.05, 0.1, 0.2, 0.4]
iters = range(20)
l = list(itertools.product(n_targets, pi_targets, iters))

In [4]:
l

[(1000, 0.05, 0),
 (1000, 0.05, 1),
 (1000, 0.05, 2),
 (1000, 0.05, 3),
 (1000, 0.05, 4),
 (1000, 0.05, 5),
 (1000, 0.05, 6),
 (1000, 0.05, 7),
 (1000, 0.05, 8),
 (1000, 0.05, 9),
 (1000, 0.05, 10),
 (1000, 0.05, 11),
 (1000, 0.05, 12),
 (1000, 0.05, 13),
 (1000, 0.05, 14),
 (1000, 0.05, 15),
 (1000, 0.05, 16),
 (1000, 0.05, 17),
 (1000, 0.05, 18),
 (1000, 0.05, 19),
 (1000, 0.1, 0),
 (1000, 0.1, 1),
 (1000, 0.1, 2),
 (1000, 0.1, 3),
 (1000, 0.1, 4),
 (1000, 0.1, 5),
 (1000, 0.1, 6),
 (1000, 0.1, 7),
 (1000, 0.1, 8),
 (1000, 0.1, 9),
 (1000, 0.1, 10),
 (1000, 0.1, 11),
 (1000, 0.1, 12),
 (1000, 0.1, 13),
 (1000, 0.1, 14),
 (1000, 0.1, 15),
 (1000, 0.1, 16),
 (1000, 0.1, 17),
 (1000, 0.1, 18),
 (1000, 0.1, 19),
 (1000, 0.2, 0),
 (1000, 0.2, 1),
 (1000, 0.2, 2),
 (1000, 0.2, 3),
 (1000, 0.2, 4),
 (1000, 0.2, 5),
 (1000, 0.2, 6),
 (1000, 0.2, 7),
 (1000, 0.2, 8),
 (1000, 0.2, 9),
 (1000, 0.2, 10),
 (1000, 0.2, 11),
 (1000, 0.2, 12),
 (1000, 0.2, 13),
 (1000, 0.2, 14),
 (1000, 0.2, 15),
 (